## Museum Cameras

This is a computational geometry problem.

Sources:
- [1] https://www.reddit.com/r/dailyprogrammer/comments/41346z/20160115_challenge_249_hard_museum_cameras/

### Question:

You run a museum, and you have a small budget - but you have to protect the museum with cameras. Given some descriptions of rooms, can you organize the smallest number of cameras to view the whole room?

Some assumptions and other factors for you to work with:

Cameras can't see around corners.
 - You can only place cameras in corners.
 - Assume every camera has a field of view of 180 degrees, yielding a semicircular field of view.
 - Assume every camera's field of view will be equal to the left and right of the line in the corner where the camera is placed; this line bisects the angle of the corner. The camera points away from the corner.
 - Assume every camera has an otherwise infinite view.

*INPUT*

You'll be given a row with a single number N that tells you how many points to read. Then on the next line you'll be given N points in a Cartesian coordinate space to draw the bounding box of the museum room. For example:
```
3
(0,0) (3,6) (6,0)
```

*OUTPUT*

Your program should emit the position of the cameras needed to cover the area. From our example:
```
(0,0)
```
That's one possible solution (for this one any of the corners would have worked).

If the shape has no solution, emit something like "The architect has no concept of security" because maybe they're collaborating with art theives.

### Solution (Thoughts...)

So I think the biggest thing to our advantage here is that this problem is constrained to 2D.

Given the FOV of the camera, and the certain placement within the center angle of a corner, we can probably calculate the coverage of the space from a single camera. This will probably involve calculating intersection points of lines? Since we can't see through corners

Then once we know the coverage of a certain camera it becomes a permutation problem about finding the least number of cameras that can fully cover the space. Noting that we have to exhaustively try out all combinations because there is the possibility of no solution.

The question offers an [article](https://www.redblobgames.com/articles/visibility/) as inspiration. It seems like raycasting from the camera's perspective is the way to go. Taking special care about corners is also important and probably a way to speed up computation.

Some basic utilities I think we will need are:
 - Intersection check between two lines (where it happens? radial distance from camera?)
 - Line conversion to polar coordinates for ray casting? Does polar make this problem easier or harder?
 - Some kind of coverage analyzer. How do you know if a space is fully covered?
 - Plotting for debugging
 
Just had a different idea. What if you went along each discretized unit of wall space and checked whether it was visible from any of the cameras.

Which just made me think about another difficulty here: outside corners. How do you know which direction the camera needs to face?

Lets write some pseudo code. Using a high to low approach (write the higher level loops then delve deeper defining the needed utilities)

```
# High to low (this is the final loop)
best_permutation = 'Not possible'
for camera_permutation in camera_permutations:
    if allvisible(camera_permutation, room) and len(camera_permutation) < best_permutation:
        best_permutation = camera_permutation
print('The best camera setup is: ', best_permutation)

# One level lower
def allvisible(camera_list, room):
    segments = wall_segments(room)
    visible_segments = []
    for camera in camera_list:
        for angle in range(camera.min_angle, camera.max_angle):
            for segment in segments:
                if intersect(camera, angle, segment):
                    segments.pop(segment)
                    visible_segments.append(segment)
    if len(segments) == 0:
        return True
    return False
```

I dont like all the nested loops in this. As well as having to discretize the walls in the room

In [ ]:
from collections import namedtuple

# Some custom tuples for this problem
Point = namedtuple('Point', ['x', 'y'])
Wall = namedtuple('Wall', ['p1', 'p2'])
Camera = namedtuple('Camera', ['p', 'min_angle', 'max_angle'])
